In [324]:
import pandas as pd
import matplotlib.pyplot as plt

In [325]:
df_location = pd.read_csv("dataset/location.csv")

In [326]:
df_cases = pd.read_csv("dataset/individual.csv")

In [327]:
df_cases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 557364 entries, 0 to 557363
Data columns (total 10 columns):
age                       260490 non-null object
sex                       263630 non-null object
province                  550796 non-null object
country                   557340 non-null object
latitude                  557362 non-null float64
longitude                 557362 non-null float64
date_confirmation         556902 non-null object
additional_information    34395 non-null object
source                    348173 non-null object
outcome                   557364 non-null object
dtypes: float64(2), object(8)
memory usage: 42.5+ MB


In [328]:
df_cases.describe()

,latitude,longitude
count,557362.000000,557362.000000
mean,18.138385,27.285771
std,20.455801,67.577194
min,-54.808030,-159.727596
25%,11.042850,-58.473080
50%,19.036810,72.834830
75%,28.456000,77.209100
max,70.071800,174.740000


In [329]:
# out of total 557,364 values 260,490 values are not NULL in age column => 296,874 missing values

In [330]:
age_col = df_cases['age']
age_col = age_col.to_frame()
age_col = age_col[age_col['age'].notna()]
age_col.head()

,age
1,21.0
2,94.0
4,2.0
5,29.0
8,35-59


In [331]:
age_col['format']= age_col['age'].str.extract(r'([0-9][0-9]-[0-9][0-9])')

a = age_col['format'].str.extract(r'([0-9][0-9])')
b = age_col['format'].str.extract(r'(-[0-9][0-9])')
b = b[0].str.extract(r'([0-9][0-9])')
b[0] = pd.to_numeric(b[0])
a[0] = pd.to_numeric(a[0])

new = round((a[0]+b[0])/2)
new = new.to_frame()
# new = new.dropna()
age_col['format'] = new[0]

In [332]:
age_col

,age,format
1,21.0,NaN
2,94.0,NaN
4,2.0,NaN
5,29.0,NaN
8,35-59,47.0
10,35-59,47.0
11,48.0,NaN
12,51.0,NaN
15,50.0,NaN
16,20-29,24.0


In [333]:
age_col['format2'] = age_col['age'].str.extract(r'([0-9]+)')
# here merge format2 into format and made format = age

age_col.format[age_col.format.isnull()] = age_col.format2
age_col.age = age_col.format
age_col

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,age,format,format2
1,21,21,21
2,94,94,94
4,2,2,2
5,29,29,29
8,47,47,35
10,47,47,35
11,48,48,48
12,51,51,51
15,50,50,50
16,24,24,20


In [334]:
type(age_col.age)
age_col.age= pd.to_numeric(age_col.age)

In [335]:
age_col.age.describe()

count    260490.000000
mean         43.701194
std          19.922561
min           0.000000
25%          28.000000
50%          44.000000
75%          57.000000
max         121.000000
Name: age, dtype: float64

In [336]:
age_col.age

1         21.0
2         94.0
4          2.0
5         29.0
8         47.0
          ... 
557352    73.0
557353    74.0
557354    65.0
557359    56.0
557362    25.0
Name: age, Length: 260490, dtype: float64